# Re-evaluación de los modelos

In [14]:
import pandas
import numpy as np
from os import path, remove
from tensorflow.keras.models import load_model
from reporter import Reporter
from csv_creator import extract_files_from_zip
from datetime import datetime


CSV_VERSION = 'V2'


def load_test_set(experiment_folder, experiment_name):
    npzfile = np.load(path.join(experiment_folder, '{}_test.npz'.format(experiment_name)))
    return npzfile['X_test'], npzfile['y_test']


def evaluate_all_models(experiment_folder, experiment_name, window_length, displacement):
    X_test, y_test = load_test_set(experiment_folder, experiment_name)
    dataframe = pandas.DataFrame(columns=['ventana', 'desplazamiento', 'id_modelo', 'TP', 'FN', 'TN', 'FP', 'MCC'])
    for model_id in np.arange(10):
        model = load_model(path.join(
            experiment_folder,
            '{}_MOD{}.h5'.format(experiment_name, model_id)
        ))
        reporter = Reporter(['normal', 'anomaly'], y_test, model.predict(X_test))
        dataframe.loc[model_id] = [window_length, displacement, model_id] + reporter.get_confusion_matrix()
    return dataframe


def evaluate_all_experiments(data_directory, configuration):
    count = 0
    skipped = 0
    for zip_file, parameters in configuration.get_experiments(): 
        window_length = parameters[-2]
        displacement  = parameters[-1]
        experiment_folder, experiment_name = extract_files_from_zip(zip_file, data_directory)
        csv_file = path.join(experiment_folder, '{}_{}.csv'.format(experiment_name, CSV_VERSION))
        if path.exists(csv_file):
            count += 1
            print('Existe {}'.format(experiment_name))
            continue        
        if displacement == 1:
            skipped += 1
            continue
        count += 1
        start = datetime.now()
        dataframe = evaluate_all_models(experiment_folder, experiment_name, window_length, displacement)
        dataframe.to_csv(csv_file, index=False, sep=';', decimal=',')
        print(
            'Hechos: {}, Ultimo tiempo (V {}, D {}): {}'.format(
                count, 
                window_length, 
                displacement, 
                (datetime.now() - start))
        )
    print('Saltados: {}'.format(skipped))



In [ ]:
from experiments_configuration import MLP2Configuration, MLP3Configuration, get_natural_prefixes, get_natural_windows

print('Re-evaluando naturales...1: {}'.format(datetime.now()))
evaluate_all_experiments(
    data_directory=path.join('2_series_naturales', 'MLP2'), 
    configuration=MLP2Configuration(get_natural_prefixes(), windows=get_natural_windows())
)
print('Re-evaluando naturales...2: {}'.format(datetime.now()))
evaluate_all_experiments(
    data_directory=path.join('2_series_naturales', 'MLP3'), 
    configuration=MLP3Configuration(get_natural_prefixes(), windows=get_natural_windows())
)
print('Re-evaluando naturales...3: {}'.format(datetime.now()))

In [ ]:
from experiments_configuration import MLP2Configuration, MLP3Configuration, get_senoidal_prefixes, get_senoidal_windows


print('Re-evaluando senoidales...1: {}'.format(datetime.now()))
evaluate_all_experiments(
    data_directory=path.join('1_series_periodicas', 'MLP2'), 
    configuration=MLP2Configuration(get_senoidal_prefixes(), windows=get_senoidal_windows())
)
print('Re-evaluando senoidales...2: {}'.format(datetime.now()))
evaluate_all_experiments(
    data_directory=path.join('1_series_periodicas', 'MLP3'), 
    configuration=MLP3Configuration(get_senoidal_prefixes(), windows=get_senoidal_windows())
)
print('Re-evaluando senoidales...3: {}'.format(datetime.now()))



# CSV senoidales

In [1]:
from os import path

def create_CSV_files_from_experiment(csv_creator, base_directory, csv_base_directory, architecture,legend_function):
    experiments_directory = path.join(base_directory, architecture)
    csv_directory = path.join(base_directory, csv_base_directory, architecture)
    
    csv_creator.combine_csv_files_from_experiment(
        experiments_directory, 
        csv_directory, 
    )
    sumup_directory = path.join(base_directory, csv_base_directory)
    sumup_filename = path.join(sumup_directory, '{}_resumen.csv'.format(architecture))
    csv_creator.sum_up_csv_files(csv_directory, sumup_filename)
    csv_creator.find_best_models(
        sumup_filename,
        path.join(sumup_directory, '{}_tabla_mejores_modelos.csv'.format(architecture)),
        legend_function
    )
    

### MLP 2 capas

In [2]:
from csv_creator import MLP2CsvCreator
from experiments_configuration import MLP2Configuration, get_senoidal_prefixes, get_senoidal_windows
from charts import get_legend_label_for_displacements_chart

create_CSV_files_from_experiment(
    MLP2CsvCreator(MLP2Configuration(get_senoidal_prefixes(), windows=get_senoidal_windows())), 
    path.join('1_series_periodicas'),
    'CSV_results',
    'MLP2',
    get_legend_label_for_displacements_chart
)


Finished !! 1800 models
Mejores modelos (promedio)... (1_series_periodicas\CSV_results\MLP2_resumen.csv)
Mejor para 1.1% anomalías, Senoidal Simple : 25 0.2 ... 0.90
Mejor para 1.1% anomalías, Senoidal Simple (ruido 2.5%): 25 0.2 ... 0.81
Mejor para 1.1% anomalías, Senoidal Simple (ruido 5%): 25 0.3 ... 0.73
Mejor para 1.1% anomalías, Senoidal Simple (ruido 10%): 25 0.2 ... 0.63
Mejor para 1.1% anomalías, Senoidal Simple (ruido 20%): 25 0.2 ... 0.48
Mejor para 2% anomalías, Senoidal Simple : 25 0.3 ... 0.80
Mejor para 2% anomalías, Senoidal Simple (ruido 2.5%): 25 0.2 ... 0.75
Mejor para 2% anomalías, Senoidal Simple (ruido 5%): 25 0.2 ... 0.70
Mejor para 2% anomalías, Senoidal Simple (ruido 10%): 25 0.2 ... 0.60
Mejor para 2% anomalías, Senoidal Simple (ruido 20%): 25 0.3 ... 0.42
Mejor para 1.1% anomalías, Senoidal Combinada : 25 0.2 ... 0.83
Mejor para 1.1% anomalías, Senoidal Combinada (ruido 2.5%): 25 0.2 ... 0.77
Mejor para 1.1% anomalías, Senoidal Combinada (ruido 5%): 25 0.3 ..

### MLP 3 capas

In [3]:
from csv_creator import MLP3CsvCreator
from experiments_configuration import MLP3Configuration, get_senoidal_prefixes, get_senoidal_windows
from charts import get_legend_label_for_displacements_chart

create_CSV_files_from_experiment(
    MLP3CsvCreator(MLP3Configuration(get_senoidal_prefixes(), windows=get_senoidal_windows())), 
    path.join('1_series_periodicas'),
    'CSV_results',
    'MLP3',
    get_legend_label_for_displacements_chart
)

Finished !! 5400 models
Mejores modelos (promedio)... (1_series_periodicas\CSV_results\MLP3_resumen.csv)
Mejor para 1.1% anomalías, Senoidal Simple : 25 15 0.2 ... 0.93
Mejor para 1.1% anomalías, Senoidal Simple (ruido 2.5%): 25 15 0.2 ... 0.86
Mejor para 1.1% anomalías, Senoidal Simple (ruido 5%): 25 15 0.2 ... 0.77
Mejor para 1.1% anomalías, Senoidal Simple (ruido 10%): 25 15 0.2 ... 0.69
Mejor para 1.1% anomalías, Senoidal Simple (ruido 20%): 25 15 0.2 ... 0.51
Mejor para 2% anomalías, Senoidal Simple : 25 15 0.3 ... 0.89
Mejor para 2% anomalías, Senoidal Simple (ruido 2.5%): 25 15 0.2 ... 0.83
Mejor para 2% anomalías, Senoidal Simple (ruido 5%): 25 15 0.3 ... 0.78
Mejor para 2% anomalías, Senoidal Simple (ruido 10%): 25 15 0.2 ... 0.67
Mejor para 2% anomalías, Senoidal Simple (ruido 20%): 25 10 0.2 ... 0.48
Mejor para 1.1% anomalías, Senoidal Combinada : 25 15 0.2 ... 0.88
Mejor para 1.1% anomalías, Senoidal Combinada (ruido 2.5%): 25 15 0.3 ... 0.82
Mejor para 1.1% anomalías, Seno

### Autoencoders

In [5]:
from csv_creator import AutoencoderCsvCreator
from experiments_configuration import AutoencoderConfiguration, get_senoidal_prefixes, get_senoidal_windows_autoencoder
from charts import get_legend_label_for_displacements_chart


for autoencoder_type in ['VAE', 'AE']:
    print('AUTOENCODER TYPE = {}'.format(autoencoder_type))
    create_CSV_files_from_experiment(
        AutoencoderCsvCreator(AutoencoderConfiguration(
            autoencoder_type,
            get_senoidal_prefixes(), 
            windows=get_senoidal_windows_autoencoder()
        )), 
        path.join('1_series_periodicas'),
        'CSV_results',
        autoencoder_type,
        get_legend_label_for_displacements_chart
    )


AUTOENCODER TYPE = VAE
Finished !! 1620 models
Mejores modelos (promedio)... (1_series_periodicas\CSV_results\VAE_resumen.csv)
Mejor para 1.1% anomalías, Senoidal Simple : 15  5 ... 0.44
Mejor para 1.1% anomalías, Senoidal Simple (ruido 2.5%): 15  5 ... 0.43
Mejor para 1.1% anomalías, Senoidal Simple (ruido 5%): 15  5 ... 0.40
Mejor para 1.1% anomalías, Senoidal Simple (ruido 10%): 20  5 ... 0.35
Mejor para 1.1% anomalías, Senoidal Simple (ruido 20%): 25 15 ... 0.30
Mejor para 2% anomalías, Senoidal Simple : 15  5 ... 0.20
Mejor para 2% anomalías, Senoidal Simple (ruido 2.5%): 15  5 ... 0.20
Mejor para 2% anomalías, Senoidal Simple (ruido 5%): 15  5 ... 0.18
Mejor para 2% anomalías, Senoidal Simple (ruido 10%): 15  5 ... 0.17
Mejor para 2% anomalías, Senoidal Simple (ruido 20%): 15 15 ... 0.12
Mejor para 1.1% anomalías, Senoidal Combinada : 15  5 ... 0.16
Mejor para 1.1% anomalías, Senoidal Combinada (ruido 2.5%): 15  5 ... 0.16
Mejor para 1.1% anomalías, Senoidal Combinada (ruido 5%):

# CSV naturales

### MLP 2 capas

In [4]:
from csv_creator import MLP2CsvCreator
from experiments_configuration import MLP2Configuration, get_natural_prefixes, get_natural_windows
from charts import build_legend_entry_for_animals

create_CSV_files_from_experiment(
    MLP2CsvCreator(MLP2Configuration(get_natural_prefixes(), windows=get_natural_windows())), 
    path.join('2_series_naturales'),
    'CSV_results',
    'MLP2',
    build_legend_entry_for_animals
)

Finished !! 540 models
Mejores modelos (promedio)... (2_series_naturales\CSV_results\MLP2_resumen.csv)
Mejor para Grillo-Ballena, 1%: 25 0.3 ... 0.58
Mejor para Grillo-Ballena, 2%: 25 0.3 ... 0.64
Mejor para Grillo-Ballena, 3%: 25 0.3 ... 0.43
Mejor para Grillo-Ballena, 5%: 25 0.3 ... 0.56
Mejor para Grillo-Ballena, 7%: 25 0.2 ... 0.86
Mejor para Grillo-Ballena, 10%: 25 0.3 ... 0.58
Mejores modelos (mejor caso)... (2_series_naturales\CSV_results\MLP2_resumen.csv)
Mejor para Grillo-Ballena, 1%: 25 0.3 ... 0.83
Mejor para Grillo-Ballena, 2%: 25 0.3 ... 0.76
Mejor para Grillo-Ballena, 3%: 20 0.2 ... 0.49
Mejor para Grillo-Ballena, 5%: 25 0.3 ... 0.63
Mejor para Grillo-Ballena, 7%: 25 0.2 ... 0.96
Mejor para Grillo-Ballena, 10%: 25 0.3 ... 0.65
Mejor para general: 25 0.3 ... 0.61


### MLP 3 capas

In [5]:
from csv_creator import MLP3CsvCreator
from experiments_configuration import MLP3Configuration, get_natural_prefixes, get_natural_windows
from charts import build_legend_entry_for_animals

create_CSV_files_from_experiment(
    MLP3CsvCreator(MLP3Configuration(get_natural_prefixes(), windows=get_natural_windows())), 
    path.join('2_series_naturales'),
    'CSV_results',
    'MLP3',
    build_legend_entry_for_animals
)

Finished !! 1620 models
Mejores modelos (promedio)... (2_series_naturales\CSV_results\MLP3_resumen.csv)
Mejor para Grillo-Ballena, 1%: 25  5 0.2 ... 0.60
Mejor para Grillo-Ballena, 2%: 25 10 0.2 ... 0.63
Mejor para Grillo-Ballena, 3%: 25 15 0.2 ... 0.47
Mejor para Grillo-Ballena, 5%: 25  5 0.3 ... 0.59
Mejor para Grillo-Ballena, 7%: 25 15 0.2 ... 0.90
Mejor para Grillo-Ballena, 10%: 25 15 0.3 ... 0.62
Mejores modelos (mejor caso)... (2_series_naturales\CSV_results\MLP3_resumen.csv)
Mejor para Grillo-Ballena, 1%: 25 15 0.2 ... 0.82
Mejor para Grillo-Ballena, 2%: 25 15 0.3 ... 0.78
Mejor para Grillo-Ballena, 3%: 25  5 0.2 ... 0.54
Mejor para Grillo-Ballena, 5%: 25 15 0.2 ... 0.65
Mejor para Grillo-Ballena, 7%: 25 10 0.2 ... 0.96
Mejor para Grillo-Ballena, 10%: 25 10 0.2 ... 0.69
Mejor para general: 25 10 0.2 ... 0.63


### Autoencoders

In [6]:
from csv_creator import AutoencoderCsvCreator
from experiments_configuration import AutoencoderConfiguration, get_natural_prefixes, get_natural_windows_autoencoder
from charts import build_legend_entry_for_animals


for autoencoder_type in ['VAE', 'AE']:
    print('AUTOENCODER TYPE = {}'.format(autoencoder_type))
    create_CSV_files_from_experiment(
        AutoencoderCsvCreator(AutoencoderConfiguration(
            autoencoder_type,
            get_natural_prefixes(), 
            windows=get_natural_windows_autoencoder()
        )), 
        path.join('2_series_naturales'),
        'CSV_results',
        autoencoder_type,
        build_legend_entry_for_animals
    )


AUTOENCODER TYPE = VAE
Finished !! 648 models
Mejores modelos (promedio)... (2_series_naturales\CSV_results\VAE_resumen.csv)
Mejor para Grillo-Ballena, 1%: 20 10 ... 0.15
Mejor para Grillo-Ballena, 2%: 25  5 ... 0.33
Mejor para Grillo-Ballena, 3%: 20 15 ... 0.14
Mejor para Grillo-Ballena, 5%: 15 10 ... 0.39
Mejor para Grillo-Ballena, 7%: 15  5 ... 0.37
Mejor para Grillo-Ballena, 10%: 25 10 ... 0.29
Mejores modelos (mejor caso)... (2_series_naturales\CSV_results\VAE_resumen.csv)
Mejor para Grillo-Ballena, 1%: 25 15 ... 0.19
Mejor para Grillo-Ballena, 2%: 25 15 ... 0.44
Mejor para Grillo-Ballena, 3%: 25 15 ... 0.22
Mejor para Grillo-Ballena, 5%: 25 15 ... 0.42
Mejor para Grillo-Ballena, 7%: 25 15 ... 0.45
Mejor para Grillo-Ballena, 10%: 25  5 ... 0.33
Mejor para general: 25 15 ... 0.28
AUTOENCODER TYPE = AE
Finished !! 648 models
Mejores modelos (promedio)... (2_series_naturales\CSV_results\AE_resumen.csv)
Mejor para Grillo-Ballena, 1%: 15 10 ... 0.62
Mejor para Grillo-Ballena, 2%: 20 10